In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('recommender').master('local[4]').getOrCreate()

23/09/17 16:47:15 WARN Utils: Your hostname, juan-MS-7A57 resolves to a loopback address: 127.0.1.1; using 192.168.20.22 instead (on interface enp6s0)
23/09/17 16:47:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/17 16:47:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Data Exploration

In [3]:
data = spark.read.csv('../data/movielens_ratings.csv', inferSchema=True, header=True)

In [4]:
data.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [5]:
data.show(5)

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
+-------+------+------+
only showing top 5 rows



In [6]:
data.count()

1501

In [7]:
data.dropna().count()

1501

In [8]:
# looks like no missing data!

In [9]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



### Create Train / Test Split

In [10]:
train, test = data.randomSplit([0.8, 0.2])

### Alternating Least Squares Model

In [12]:
from pyspark.ml.recommendation import ALS

In [13]:
als = ALS(
    maxIter=5, 
    regParam=0.01, 
    userCol='userId', 
    itemCol='movieId', 
    ratingCol='rating')

In [14]:
model = als.fit(train)

In [15]:
predictions = model.transform(test)

In [16]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      2|   4.0|    28|  2.1348538|
|      0|   1.0|    26|   1.084711|
|      3|   1.0|    26|  1.4554554|
|      4|   1.0|    12|  3.1881073|
|      0|   1.0|    22|  1.1590129|
|      0|   1.0|    13|  0.9261657|
|      2|   1.0|    16| 0.45263612|
|      5|   1.0|     6|  1.3981816|
|      2|   1.0|     3|  2.0739524|
|      1|   1.0|    20|  0.5410995|
|      0|   1.0|     5|-0.22031544|
|      4|   1.0|    23|  3.7625399|
|      4|   1.0|     7|  3.1864507|
|      3|   1.0|    29|  0.9779828|
|      2|   4.0|    21|  3.6768405|
|      3|   1.0|    21|  2.1798239|
|      0|   1.0|    11|  0.8300039|
|      3|   3.0|    14|  0.4794108|
|      4|   1.0|    14|  2.9792824|
|      2|   3.0|     0|  1.3998227|
+-------+------+------+-----------+
only showing top 20 rows



### Evaluatin Test Predictions

In [17]:
from pyspark.ml.evaluation import RegressionEvaluator

In [18]:
evaluator = RegressionEvaluator(
    metricName='rmse', 
    labelCol='rating', 
    predictionCol='prediction')

In [19]:
rmse = evaluator.evaluate(predictions)

In [20]:
print('RMSE',rmse)

RMSE 1.8410205854877306


In [21]:
# looks like we are about 2 stars off on a 5 star rating set
# pretty poor results, but this data set is small

### Generate Recommendations for Single User

In [22]:
single_user = test.filter('userId = 11').select('movieId', 'userId')

In [23]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|     12|    11|
|     13|    11|
|     16|    11|
|     27|    11|
|     45|    11|
|     72|    11|
|     76|    11|
|     86|    11|
|     88|    11|
+-------+------+



In [24]:
recommendations = model.transform(single_user)

In [25]:
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     13|    11|  4.265451|
|     16|    11| 3.7958543|
|     88|    11| 3.7709146|
|     27|    11|  1.740818|
|     72|    11| 1.5834702|
|     86|    11| 1.4703176|
|     76|    11| 1.1800952|
|     45|    11| 1.0255547|
|      0|    11| 0.8300039|
|     12|    11|0.38264716|
+-------+------+----------+

